
# Pyteomics Tutorial

Import the following modules into your notebook. If you get an error, try running `pip install {name of the missing module}`.

In [23]:
import pandas as pd
import numpy as np
import os
import pyteomics.mzml
import spectrum_utils.spectrum as sus
from pathlib import Path

#### Accessing the mzML file
In order to access the mzML file, we need to give python its absolute address. In Hannah's tutorial she used `os.path.abspath(os.path.dirname(__file__))` to obtain the absolute address to the python script. Unfortunately `__file__` does not exist in JupyterNotebooks so I just copy and pasted the absolute address of my Notebook.

In [24]:
mzML_file_name = "Ex_Auto_J3_30umTB_2ngQC_60m_1.mzML"
path_to_pyteomics_tutorial = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev"
complete_path_to_mzml = os.path.join(path_to_pyteomics_tutorial, mzML_file_name)

Once we have the complete file path we can create an `MzML` object using `pyteomics.mzml`

In [25]:
mzml_practice = pyteomics.mzml.MzML(complete_path_to_mzml)

#### Parsing through the MzML object
In Hannah's data_loader script she created a helper function to pull out relevant information from the MzML object. This function allows us to pull out the data corresponding to one scan at a time. 


In [26]:
def mzml_helper(scan, mzml):
    # mzML files are organized by Mass Spec scan number so we will begin by creating an ID with the scan number
    my_id = 'controllerType=0 controllerNumber=1 scan='+ str(scan)

    # Using this ID we will create a dictionary containing data for this scan
    # Note: The keys in this dictionary are the controlled variable names and the values are the data associated with 
    #       these variables.
    spectrum_dict = mzml.get_by_id(my_id)

    # check that the scan is a MS2 spectra
    if spectrum_dict['ms level'] != 2:
        print('This is a MS1 scan. The information will not be processed.')
        return

    # Now we will start pulling out relevant data and storing it in local variables
    spectrum_id = spectrum_dict['id']

    # Information nested within the mzML document is stored in nested lists within the spectrum_dict. Therefore to 
    # access the retention_time value, we must look at the variable 'scan start time' within the 0th element of
    # the 'scan' list which is located within 'scanList'. Note that the indentation within the mzML file along with
    # the <[variable_name]> indicates the organization of the nested information. See below for an example of this 
    # nesting.
    retention_time = (spectrum_dict['scanList']['scan'][0].get('scan start time', -1))
    mz_array = list(spectrum_dict['m/z array'])
    intensity_array = list(spectrum_dict['intensity array'])

    # Precursor Information
    # This section is dedicated to obtaining the information from the precursor or MS1 spectrum.
    if 'precursorList' in spectrum_dict.keys():
        precursor = spectrum_dict['precursorList']['precursor'][0]
        precursor_ion = precursor['selectedIonList']['selectedIon'][0]
        precursor_mz = precursor_ion['selected ion m/z']
        if 'peak intensity' in precursor_ion:
            precursor_intensity =  precursor_ion['peak intensity']
        else:
            precursor_intensity = None
        if 'charge state' in precursor_ion:
            precursor_charge = int(precursor_ion['charge state'])
        elif 'possible charge state' in precursor_ion:
            precursor_charge = int(precursor_ion['possible charge state'])
        else:
            precursor_charge = 'NAN'
    else:
        precursor_intensity = None
        
    # Create an array with the mz_array, the intensity array, and the precursor_intensity
    all_info = [mz_array,intensity_array,precursor_intensity]

    return all_info

def printDictionary(myDict):
    for key in myDict.keys():
        print(f"{key}: {myDict[key]}")

```<scanList count="1">
      <cvParam cvRef="MS" accession="MS:1000795" name="no combination" value=""/>
      <scan>
        <cvParam cvRef="MS" accession="MS:1000016" name="scan start time" value="0.002329668967" unitCvRef="UO" unitAccession="UO:0000031" unitName="minute"/>
        <cvParam cvRef="MS" accession="MS:1000512" name="filter string" value="FTMS + p NSI Full ms [375.0000-1575.0000]"/>
        <cvParam cvRef="MS" accession="MS:1000616" name="preset scan configuration" value="1"/>
        <cvParam cvRef="MS" accession="MS:1000927" name="ion injection time" value="21.280875429511" unitCvRef="UO" unitAccession="UO:0000028" unitName="millisecond"/>
        <scanWindowList count="1">
          <scanWindow>
            <cvParam cvRef="MS" accession="MS:1000501" name="scan window lower limit" value="375.0" unitCvRef="MS" unitAccession="MS:1000040" unitName="m/z"/>
            <cvParam cvRef="MS" accession="MS:1000500" name="scan window upper limit" value="1575.0" unitCvRef="MS" unitAccession="MS:1000040" unitName="m/z"/>
          </scanWindow>
        </scanWindowList>
      </scan>
    </scanList>```

In [27]:
mzml_info = mzml_helper(5,mzml_practice)
mzml_info

[[134.21812438964844,
  140.13174438476562,
  199.84735107421875,
  343.6039733886719,
  348.05560302734375,
  387.7862854003906,
  524.0426025390625,
  706.332275390625,
  716.4232177734375,
  1256.611328125,
  1540.68505859375,
  2416.969482421875,
  3045.887939453125,
  3229.28759765625],
 [672.3894653320312,
  728.9189453125,
  797.3052978515625,
  926.0078735351562,
  802.8884887695312,
  827.426025390625,
  733.2654418945312,
  794.1657104492188,
  863.4955444335938,
  874.4263916015625,
  881.8046264648438,
  796.6223754882812,
  1042.2738037109375,
  950.6578979492188],
 None]

## Working with the PSM files

In [28]:
# Let's get the file path for the psm files

# File names
peptides_file_name = "Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_Peptides.psmtsv"
proteins_file_name = "Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_ProteinGroups.tsv"

# Complete file paths
peptides_complete_file_path = os.path.join(path_to_pyteomics_tutorial, peptides_file_name)
proteins_complete_file_path = os.path.join(path_to_pyteomics_tutorial, proteins_file_name)

# Using pandas to generate databases from these csv files
peptide_dataframe = pd.read_table(peptides_complete_file_path, delimiter='\t')
protein_dataframe = pd.read_table(proteins_complete_file_path, delimiter='\t')


In [29]:
# View the first 5 rows of the peptide_dataframe
peptide_dataframe.head()

,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,3.352519e+05,16649,2.0,1280.62784,2559.24113,28.510,...,,,1,0,0.0,1,0,0.0,0.000063,0.000026
1,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,27140,77.59262,155.0,1.258275e+06,27124,2.0,1108.03646,2214.05838,26.603,...,,,2,0,0.0,2,0,0.0,0.000029,0.000014
2,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,32708,91.05715,133.0,3.061968e+05,32695,3.0,832.05256,2493.13585,25.433,...,,,3,0,0.0,3,0,0.0,0.000009,0.000005
3,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,33439,92.82384,200.0,8.794933e+05,33418,3.0,856.05885,2565.15472,25.426,...,,,4,0,0.0,4,0,0.0,0.000002,0.000001
4,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,13306,44.50553,165.0,4.429513e+05,13293,3.0,729.02008,2184.03843,25.266,...,,,5,0,0.0,5,0,0.0,0.000046,0.000021


In [30]:
# View the first 5 rows of the protein_dataframe
protein_dataframe.head()

,Protein Accession,Gene,Organism,Protein Full Name,Protein Unmodified Mass,Number of Proteins in Group,Unique Peptides,Shared Peptides,Number of Peptides,Number of Unique Peptides,...,Sequence Coverage with Mods,Modification Info List,Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,Number of PSMs,Protein Decoy/Contaminant/Target,Protein Cumulative Target,Protein Cumulative Decoy,Protein QValue,Best Peptide Score,Best Peptide Notch QValue
P10809,HSPD1,Homo sapiens,"60 kDa heat shock protein, mitochondrial",61016.3817425066,1,NaN,NaN,24,19,0.50611,...,NaN,8.918622e+06,63,T,1,0,0.0,28.510203,0.0,NaN
P60709|P63261,ACTB|ACTG1,Homo sapiens,"Actin, cytoplasmic 1|Actin, cytoplasmic 2",41709.73014332879|41765.79274358662,2,NaN,NaN,18,0,0.52267|0.52267,...,NaN,7.577315e+07,99,T,2,0,0.0,26.602575,0.0,NaN
P14618,PKM,Homo sapiens,Pyruvate kinase PKM,57900.02335733301,1,NaN,NaN,28,20,0.60829,...,NaN,2.030516e+07,76,T,3,0,0.0,25.432943,0.0,NaN
P29692,EEF1D,Homo sapiens,Elongation factor 1-delta,31102.78320902701,1,NaN,NaN,11,8,0.51601,...,NaN,2.230951e+06,16,T,4,0,0.0,25.265928,0.0,NaN
P80723,BASP1,Homo sapiens,Brain acid soluble protein 1,22680.0117538596,1,NaN,NaN,13,13,0.70925,...,NaN,4.189656e+06,22,T,5,0,0.0,24.456898,0.0,NaN


In [31]:
# To make the peptide_dataframe more accessible, we will rename two of the columns
# Note that we use axis=1 to specify that we are renaming columns
peptide_dataframe = peptide_dataframe.rename({"Scan Number": "scan", "Full Sequence": "peptide"}, axis=1)

# Next we will sort the peptide_dataframe by QValue
peptide_dataframe = peptide_dataframe.sort_values("QValue")

# We will then delete duplicate scans, keeping only the first one
peptide_dataframe = peptide_dataframe.drop_duplicates(subset=["scan"], keep="first")

peptide_dataframe.head()

,File Name,scan,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,335251.91071,16649,2.0,1280.62784,2559.24113,28.510,...,,,1,0,0.0,1,0,0.0,0.000063,0.000026
1227,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,20217,60.99236,49.0,76071.94836,20203,3.0,800.73355,2399.17882,13.388,...,,,1228,0,0.0,1224,0,0.0,0.000108,0.000042
1226,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,21843,64.88158,77.0,185690.32166,21831,3.0,515.26251,1542.76569,13.388,...,,,1227,0,0.0,1223,0,0.0,0.000031,0.000015
1225,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16915,53.10976,99.0,224170.22906,16902,3.0,506.92071,1517.74029,13.390,...,,,1226,0,0.0,1222,0,0.0,0.000013,0.000007
1224,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,19666,59.67404,72.0,282899.35223,19653,2.0,721.90597,1441.79739,13.390,...,,,1225,0,0.0,1221,0,0.0,0.001575,0.000259


In [32]:
# Convert the scan values into numerical values
peptide_dataframe[["scan"]] = peptide_dataframe[["scan"]].apply(pd.to_numeric)

# Convert the "Scan Retention Time" series into strings
peptide_dataframe["Scan Retention Time"] = peptide_dataframe["Scan Retention Time"].astype(str)

# Create a new column called "temp_minute" 
# The values of this column will be a list created by splitting the "Scan Retention Time"
# values at the ".", this will give us values of this format: [num_minutes, num_seconds]
peptide_dataframe["temp_minute"] = peptide_dataframe["Scan Retention Time"].str.split("\.")
peptide_dataframe.head()

,File Name,scan,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue,temp_minute
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,335251.91071,16649,2.0,1280.62784,2559.24113,28.510,...,,1,0,0.0,1,0,0.0,0.000063,0.000026,"[52, 52059]"
1227,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,20217,60.99236,49.0,76071.94836,20203,3.0,800.73355,2399.17882,13.388,...,,1228,0,0.0,1224,0,0.0,0.000108,0.000042,"[60, 99236]"
1226,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,21843,64.88158,77.0,185690.32166,21831,3.0,515.26251,1542.76569,13.388,...,,1227,0,0.0,1223,0,0.0,0.000031,0.000015,"[64, 88158]"
1225,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16915,53.10976,99.0,224170.22906,16902,3.0,506.92071,1517.74029,13.390,...,,1226,0,0.0,1222,0,0.0,0.000013,0.000007,"[53, 10976]"
1224,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,19666,59.67404,72.0,282899.35223,19653,2.0,721.90597,1441.79739,13.390,...,,1225,0,0.0,1221,0,0.0,0.001575,0.000259,"[59, 67404]"


In [33]:
# Now we'll create a minutes column using the value in the 0th index of the 
# "temp_minute" column
peptide_dataframe.loc[:, 'minute'] = peptide_dataframe['temp_minute'].map(lambda x: x[0])

# Then let's convert the values of this column into integers
peptide_dataframe[["minute"]] = peptide_dataframe[["minute"]].apply(pd.to_numeric)

# Then we can drop/delete the "temp_minute" column
peptide_dataframe = peptide_dataframe.drop(columns='temp_minute')

peptide_dataframe.head()

,File Name,scan,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue,minute
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,335251.91071,16649,2.0,1280.62784,2559.24113,28.510,...,,1,0,0.0,1,0,0.0,0.000063,0.000026,52
1227,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,20217,60.99236,49.0,76071.94836,20203,3.0,800.73355,2399.17882,13.388,...,,1228,0,0.0,1224,0,0.0,0.000108,0.000042,60
1226,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,21843,64.88158,77.0,185690.32166,21831,3.0,515.26251,1542.76569,13.388,...,,1227,0,0.0,1223,0,0.0,0.000031,0.000015,64
1225,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16915,53.10976,99.0,224170.22906,16902,3.0,506.92071,1517.74029,13.390,...,,1226,0,0.0,1222,0,0.0,0.000013,0.000007,53
1224,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,19666,59.67404,72.0,282899.35223,19653,2.0,721.90597,1441.79739,13.390,...,,1225,0,0.0,1221,0,0.0,0.001575,0.000259,59


Now let's append data from the mzML file onto this peptide dataframe.
Before we do this we'll need to modify our mzml_helper function to take a row instead of a scan number.

In [34]:
def mzml_helper(row, mzml):
    # Access the value of scan for the given row
    scan = str(row['scan'])

    my_id = 'controllerType=0 controllerNumber=1 scan='+ str(scan)
    spectrum_dict = mzml.get_by_id(my_id)

    if spectrum_dict['ms level'] != 2:
        print('This is a MS1 scan. The information will not be processed.')
        return

    spectrum_id = spectrum_dict['id']
    retention_time = (spectrum_dict['scanList']['scan'][0].get('scan start time', -1))
    mz_array = list(spectrum_dict['m/z array'])
    intensity_array = list(spectrum_dict['intensity array'])

    if 'precursorList' in spectrum_dict.keys():
        precursor = spectrum_dict['precursorList']['precursor'][0]
        precursor_ion = precursor['selectedIonList']['selectedIon'][0]
        precursor_mz = precursor_ion['selected ion m/z']
        if 'peak intensity' in precursor_ion:
            precursor_intensity =  precursor_ion['peak intensity']
        else:
            precursor_intensity = None
        if 'charge state' in precursor_ion:
            precursor_charge = int(precursor_ion['charge state'])
        elif 'possible charge state' in precursor_ion:
            precursor_charge = int(precursor_ion['possible charge state'])
        else:
            precursor_charge = 'NAN'
    else:
        precursor_intensity = None
        
    all_info = [mz_array,intensity_array,precursor_intensity]

    return all_info

In [35]:
# Let's create a column named 'mzml_info' that will contain [mz_array,intensity_array,precursor_intensity]
# Note that inside the parenthesis we are running each row through the mzml_helper function
peptide_dataframe['mzml_info'] = peptide_dataframe.apply(lambda row: mzml_helper(row, mzml_practice),axis=1)

# From the 'mzml-info' column we will create three new columns and delete the 'mzml_info' column
peptide_dataframe[['mz_array', 'intensity_array', 'precursor_intensity']] = pd.DataFrame(peptide_dataframe['mzml_info'].tolist(), index= peptide_dataframe.index)
peptide_dataframe = peptide_dataframe.drop(columns='mzml_info')

In [36]:
peptide_dataframe.head()

,File Name,scan,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue,minute,mz_array,intensity_array,precursor_intensity
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,335251.91071,16649,2.0,1280.62784,2559.24113,28.510,...,0.0,1,0,0.0,0.000063,0.000026,52,"[118.0188980102539, 124.82211303710938, 175.11...","[813.1907348632812, 718.8142700195312, 5288.52...",106894.523438
1227,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,20217,60.99236,49.0,76071.94836,20203,3.0,800.73355,2399.17882,13.388,...,0.0,1224,0,0.0,0.000108,0.000042,60,"[129.102294921875, 131.43809509277344, 136.076...","[923.3621215820312, 662.60546875, 4630.6298828...",53405.894531
1226,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,21843,64.88158,77.0,185690.32166,21831,3.0,515.26251,1542.76569,13.388,...,0.0,1223,0,0.0,0.000031,0.000015,64,"[113.53097534179688, 120.08123779296875, 129.0...","[846.5783081054688, 1549.9730224609375, 660.12...",88953.414062
1225,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16915,53.10976,99.0,224170.22906,16902,3.0,506.92071,1517.74029,13.390,...,0.0,1222,0,0.0,0.000013,0.000007,53,"[110.07168579101562, 110.07616424560547, 116.0...","[7146.0556640625, 916.3556518554688, 1682.3347...",91471.835938
1224,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,19666,59.67404,72.0,282899.35223,19653,2.0,721.90597,1441.79739,13.390,...,0.0,1221,0,0.0,0.001575,0.000259,59,"[127.87828063964844, 129.10279846191406, 130.0...","[902.3700561523438, 2441.395751953125, 1403.50...",138897.890625
